In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

46373


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [15]:
add_histone = (
    "INSERT INTO histone "
    "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
    "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
)
add_histone_description = (
    "INSERT INTO histone_description "
    "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

In [7]:
data_histone_description = {
    "summary": "",
    "taxonomy": "",
    "genes": "",
    "evolution": "",
    "expression": "",
    "knock_out": "",
    "function": "",
    "sequence": "",
    "localization": "",
    "deposition": "",
    "structure": "",
    "interactions": "",
    "disease": "",
    "caveats": "",
}

In [9]:
query = "SELECT * FROM alternative_name " "WHERE name='cid'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,65,cid,None,None,None,cenH3
1,93,cid,None,None,None,cenH3_(Animals)
2,99,cid,None,None,None,cenH3_(Insecta)
3,109,cid,None,None,None,cenH3_(Drosophilidae)


# To Do H4

## <span style="color:green">Add a new nodes cH4_(Animals), cH4_(Plants), cH4_(Fungi), cH4_(Protists) to cH4</span>

## <span style="color:green">Add a new node cH4_(Vertebrata) to cH4_(Animals)</span>

## <span style="color:green">Add a new node cH4_(Mammalia) to cH4_(Vertebrata)</span>

## <span style="color:green">Change parent node of cH4_(Homo_sapiens) to cH4_(Mammalia)</span>

## <span style="color:green">Add a new node H4.V_(Trypanosomatidae) to H4</span>

## <span style="color:green">Add description to H4.V_(Trypanosomatidae)</span>

### <span style="color:green">Add summary</span>

```H4.V_(Trypanosomatidae) is a histone variant characterized so far in some trypanosomatids (T. brucei and T. cruzi). It shares ~98% sequence identity with canonical H4. H4.V co-localizes with the histone variant H3.V and DNA base J at transcription termination sites (TTSs), where it is a key factor in ensuring efficient transcription termination [kim_genetic_2021].```

### <span style="color:green">Add gene</span>

```H4.V encoded by a single-copy gene located apart from the typical  30 tandem arrays of canonical histone H4 [roson_new_2025].```

### <span style="color:green">Add function</span>

```H4.V is the major signal for RNA polymerase II transcription termination at TTSs and, together with H3.V, plays a role in repressing transcription of telomeric variant surface glycoprotein genes [kim_genetic_2021]. The enrichment of H4.V at telomeres and TTSs, coupled with its increased expression in metacyclic forms, suggests its role in regulating the expression of virulence genes and adaptation to different host environments [roson_new_2025].```

### <span style="color:green">Add knock_out</span>

```The H4.V knockout in T. brucei is viable, and the knockout cells grow normally without morphological abnormalities [kim_genetic_2021]. However, these mutants exhibit a significant increase in antisense transcription. The simultaneous knockout of H4.V and H3.V leads to severe defects in growth, DNA replication, and cell-cycle progression, which are exacerbated in the triple knockout with base J, indicating a synthetic lethal genetic interaction among these three epigenetic marks [kim_genetic_2021].```

### <span style="color:green">Add expression</span>

```H4.V expression analysis in T. cruzi demonstrated that H4.V transcript levels remain stable throughout the cell cycle, while protein abundance increases during the G2/M phase [roson_new_2025]. H4.V is more abundant in the nuclei of metacyclic trypomastigotes but is barely detectable in amastigotes and bloodstream trypomastigotes. During metacyclogenesis, its expression progressively increases, suggesting a role in parasite differentiation [roson_new_2025].```

### <span style="color:green">Add localization</span>

```While the core function of H4.V in marking transcription termination sites is conserved in trypanosomes, its genomic localization exhibits significant species-specificity between T. brucei and T. cruzi. In T. brucei, H4.V is specifically localized at transcription termination sites that demarcate polycistronic transcription units and is absent from telomeric repeats, where the histone variant H3.V and base J are instead enriched [kim_genetic_2021]. In contrast, T. cruzi H4.V displays a unique localization pattern, occupying not only convergent strand-switch regions but also telomeric regions and specific tDNA loci [roson_new_2025].```

## <span style="color:green">Add description to cH4_(Animals)</span>

### <span style="color:green">Add summary</span>

```cH4_(Animals) clustered H4 histones in animals (Metazoa), often called canonical, replication-dependent, replication-coupled or "bulk" H4 histones are a major class of H4 histones in animals with high sequence similarity across all animal species.```

### <span style="color:green">Add sequence</span>

```The threonine 71 position of histone H4 is highly conserved across numerous animal species and can be subjected to O-GlcNAcylation (H4T71Gc). This modification plays a critical role, in concert with H3K9me3, in the progression and maintenance of condensed chromatin [hayakawa_novel_2025].```

# To Do H2B

## <span style="color:green">Add a alternate name H2B.E to cH2B.E_(Mus_musculus)</span>

# DONE

## <span style="color:black">Add a new nodes cH4_(Animals), cH4_(Plants), cH4_(Fungi), cH4_(Protists) to cH4</span>

In [10]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH4'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4,variant_group,Eukaryotes,2759,59,H4,59,Canonical histones are replication-dependent w...,null,null,...,null,null,null,null,null,null,null,null,null,None


In [11]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='cH4'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Homo_sapiens),variant,Homo sapiens,9606,109,cH4,109,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [16]:
data_histone = {
    "id": "cH4_(Animals)",
    "level": "variant",
    "taxonomic_span": "Metazoa",
    "taxonomic_span_id": "33208",
    "description": None,
    "parent": "cH4",
}
cursor.execute(add_histone, data_histone)

data_histone = {
    "id": "cH4_(Plants)",
    "level": "variant",
    "taxonomic_span": "Viridiplantae",
    "taxonomic_span_id": "33090",
    "description": None,
    "parent": "cH4",
}
cursor.execute(add_histone, data_histone)

data_histone = {
    "id": "cH4_(Fungi)",
    "level": "variant",
    "taxonomic_span": "Fungi",
    "taxonomic_span_id": "4751",
    "description": None,
    "parent": "cH4",
}
cursor.execute(add_histone, data_histone)

data_histone = {
    "id": "cH4_(Protists)",
    "level": "variant",
    "taxonomic_span": "SAR,Metamonada,Discoba,Amoebozoa",
    "taxonomic_span_id": "2698737,2611341,2611352,554915",
    "description": None,
    "parent": "cH4",
}
cursor.execute(add_histone, data_histone)

In [17]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='cH4'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Animals),variant,Metazoa,33208,NaN,cH4,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,cH4_(Fungi),variant,Fungi,4751,NaN,cH4,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,cH4_(Homo_sapiens),variant,Homo sapiens,9606,109.0,cH4,109.0,null,null,null,...,null,null,null,null,null,null,null,null,null,None
3,cH4_(Plants),variant,Viridiplantae,33090,NaN,cH4,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,cH4_(Protists),variant,"SAR,Metamonada,Discoba,Amoebozoa","2698737,2611341,2611352,554915",NaN,cH4,NaN,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [18]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add a new node cH4_(Vertebrata) to cH4_(Animals)</span>

In [19]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='cH4_(Animals)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [20]:
data_histone = {
    "id": "cH4_(Vertebrata)",
    "level": "variant",
    "taxonomic_span": "Vertebrata",
    "taxonomic_span_id": "7742",
    "description": None,
    "parent": "cH4_(Animals)",
}
cursor.execute(add_histone, data_histone)

In [21]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='cH4_(Animals)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Vertebrata),variant,Vertebrata,7742,None,cH4_(Animals),None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [22]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add a new node cH4_(Mammalia) to cH4_(Vertebrata)</span>

In [23]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='cH4_(Vertebrata)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [24]:
data_histone = {
    "id": "cH4_(Mammalia)",
    "level": "variant",
    "taxonomic_span": "Mammalia",
    "taxonomic_span_id": "40674",
    "description": None,
    "parent": "cH4_(Vertebrata)",
}
cursor.execute(add_histone, data_histone)

In [25]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='cH4_(Vertebrata)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Mammalia),variant,Mammalia,40674,None,cH4_(Vertebrata),None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [26]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Change parent node of cH4_(Homo_sapiens) to cH4_(Mammalia)</span>

In [27]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='cH4_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [28]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH4_(Homo_sapiens)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Homo_sapiens),variant,Homo sapiens,9606,109,cH4,109,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [29]:
query = f"UPDATE histone SET parent='cH4_(Mammalia)' WHERE id='cH4_(Homo_sapiens)'"
cursor.execute(query)

In [30]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.parent='cH4_(Mammalia)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Homo_sapiens),variant,Homo sapiens,9606,109,cH4_(Mammalia),109,null,null,null,...,null,null,null,null,null,null,null,null,null,None


In [31]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add a new node H4.V_(Trypanosomatidae) to H4</span>

In [32]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4.V_(Trypanosomatidae)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations


In [33]:
data_histone = {
    "id": "H4.V_(Trypanosomatidae)",
    "level": "variant_group",
    "taxonomic_span": "Trypanosomes",
    "taxonomic_span_id": "93954",
    "description": None,
    "parent": "H4",
}
cursor.execute(add_histone, data_histone)

In [34]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4.V_(Trypanosomatidae)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H4.V_(Trypanosomatidae),variant_group,Trypanosomes,93954,None,H4,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [35]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add description to H4.V_(Trypanosomatidae)</span>

### <span style="color:black">Add summary</span>

```H4.V_(Trypanosomatidae) is a histone variant characterized so far in some trypanosomatids (T. brucei and T. cruzi). It shares ~98% sequence identity with canonical H4. H4.V co-localizes with the histone variant H3.V and DNA base J at transcription termination sites (TTSs), where it is a key factor in ensuring efficient transcription termination [kim_genetic_2021].```

### <span style="color:black">Add gene</span>

```H4.V encoded by a single-copy gene located apart from the typical  30 tandem arrays of canonical histone H4 [roson_new_2025].```

### <span style="color:black">Add function</span>

```H4.V is the major signal for RNA polymerase II transcription termination at TTSs and, together with H3.V, plays a role in repressing transcription of telomeric variant surface glycoprotein genes [kim_genetic_2021]. The enrichment of H4.V at telomeres and TTSs, coupled with its increased expression in metacyclic forms, suggests its role in regulating the expression of virulence genes and adaptation to different host environments [roson_new_2025].```

### <span style="color:black">Add knock_out</span>

```The H4.V knockout in T. brucei is viable, and the knockout cells grow normally without morphological abnormalities [kim_genetic_2021]. However, these mutants exhibit a significant increase in antisense transcription. The simultaneous knockout of H4.V and H3.V leads to severe defects in growth, DNA replication, and cell-cycle progression, which are exacerbated in the triple knockout with base J, indicating a synthetic lethal genetic interaction among these three epigenetic marks [kim_genetic_2021].```

### <span style="color:black">Add expression</span>

```H4.V expression analysis in T. cruzi demonstrated that H4.V transcript levels remain stable throughout the cell cycle, while protein abundance increases during the G2/M phase [roson_new_2025]. H4.V is more abundant in the nuclei of metacyclic trypomastigotes but is barely detectable in amastigotes and bloodstream trypomastigotes. During metacyclogenesis, its expression progressively increases, suggesting a role in parasite differentiation [roson_new_2025].```

### <span style="color:black">Add localization</span>

```While the core function of H4.V in marking transcription termination sites is conserved in trypanosomes, its genomic localization exhibits significant species-specificity between T. brucei and T. cruzi. In T. brucei, H4.V is specifically localized at transcription termination sites that demarcate polycistronic transcription units and is absent from telomeric repeats, where the histone variant H3.V and base J are instead enriched [kim_genetic_2021]. In contrast, T. cruzi H4.V displays a unique localization pattern, occupying not only convergent strand-switch regions but also telomeric regions and specific tDNA loci [roson_new_2025].```

In [36]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4.V_(Trypanosomatidae)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H4.V_(Trypanosomatidae),variant_group,Trypanosomes,93954,None,H4,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [37]:
data_histone_description = {
    "summary": "H4.V_(Trypanosomatidae) is a histone variant characterized so far in some trypanosomatids (T. brucei and T. cruzi). It shares ~98% sequence identity with canonical H4. H4.V co-localizes with the histone variant H3.V and DNA base J at transcription termination sites (TTSs), where it is a key factor in ensuring efficient transcription termination [kim_genetic_2021].",
    "taxonomy": None,
    "genes": "H4.V encoded by a single-copy gene located apart from the typical  30 tandem arrays of canonical histone H4 [roson_new_2025].",
    "evolution": None,
    "expression": "H4.V expression analysis in T. cruzi demonstrated that H4.V transcript levels remain stable throughout the cell cycle, while protein abundance increases during the G2/M phase [roson_new_2025]. H4.V is more abundant in the nuclei of metacyclic trypomastigotes but is barely detectable in amastigotes and bloodstream trypomastigotes. During metacyclogenesis, its expression progressively increases, suggesting a role in parasite differentiation [roson_new_2025].",
    "knock_out": "The H4.V knockout in T. brucei is viable, and the knockout cells grow normally without morphological abnormalities [kim_genetic_2021]. However, these mutants exhibit a significant increase in antisense transcription. The simultaneous knockout of H4.V and H3.V leads to severe defects in growth, DNA replication, and cell-cycle progression, which are exacerbated in the triple knockout with base J, indicating a synthetic lethal genetic interaction among these three epigenetic marks [kim_genetic_2021].",
    "function": "H4.V is the major signal for RNA polymerase II transcription termination at TTSs and, together with H3.V, plays a role in repressing transcription of telomeric variant surface glycoprotein genes [kim_genetic_2021]. The enrichment of H4.V at telomeres and TTSs, coupled with its increased expression in metacyclic forms, suggests its role in regulating the expression of virulence genes and adaptation to different host environments [roson_new_2025].",
    "sequence": None,
    "localization": "While the core function of H4.V in marking transcription termination sites is conserved in trypanosomes, its genomic localization exhibits significant species-specificity between T. brucei and T. cruzi. In T. brucei, H4.V is specifically localized at transcription termination sites that demarcate polycistronic transcription units and is absent from telomeric repeats, where the histone variant H3.V and base J are instead enriched [kim_genetic_2021]. In contrast, T. cruzi H4.V displays a unique localization pattern, occupying not only convergent strand-switch regions but also telomeric regions and specific tDNA loci [roson_new_2025].",
    "deposition": None,
    "structure": None,
    "interactions": None,
    "disease": None,
    "caveats": None,
}
cursor.execute(add_histone_description, tuple(data_histone_description.values()))

histone_description_id = cursor.lastrowid
query = (
    f"UPDATE histone SET description={histone_description_id} WHERE id='H4.V_(Trypanosomatidae)'"
)
cursor.execute(query)

In [38]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='H4.V_(Trypanosomatidae)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,H4.V_(Trypanosomatidae),variant_group,Trypanosomes,93954,256,H4,256,H4.V_(Trypanosomatidae) is a histone variant c...,None,H4.V encoded by a single-copy gene located apa...,...,"The H4.V knockout in T. brucei is viable, and ...",H4.V is the major signal for RNA polymerase II...,None,While the core function of H4.V in marking tra...,None,None,None,None,None,None


In [39]:
# Make sure data is committed to the database
conn.commit()

## <span style="color:black">Add description to cH4_(Animals)</span>

### <span style="color:black">Add summary</span>

```cH4_(Animals) clustered H4 histones in animals (Metazoa), often called canonical, replication-dependent, replication-coupled or "bulk" H4 histones are a major class of H4 histones in animals with high sequence similarity across all animal species.```

### <span style="color:black">Add sequence</span>

```The threonine 71 position of histone H4 is highly conserved across numerous animal species and can be subjected to O-GlcNAcylation (H4T71Gc). This modification plays a critical role, in concert with H3K9me3, in the progression and maintenance of condensed chromatin [hayakawa_novel_2025].```

In [40]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH4_(Animals)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Animals),variant,Metazoa,33208,None,cH4,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [42]:
data_histone_description = {
    "summary": 'cH4_(Animals) clustered H4 histones in animals (Metazoa), often called canonical, replication-dependent, replication-coupled or "bulk" H4 histones are a major class of H4 histones in animals with high sequence similarity across all animal species.',
    "taxonomy": None,
    "genes": None,
    "evolution": None,
    "expression": None,
    "knock_out": None,
    "function": None,
    "sequence": "The threonine 71 position of histone H4 is highly conserved across numerous animal species and can be subjected to O-GlcNAcylation (H4T71Gc). This modification plays a critical role, in concert with H3K9me3, in the progression and maintenance of condensed chromatin [hayakawa_novel_2025].",
    "localization": None,
    "deposition": None,
    "structure": None,
    "interactions": None,
    "disease": None,
    "caveats": None,
}
cursor.execute(add_histone_description, tuple(data_histone_description.values()))

histone_description_id = cursor.lastrowid
query = (
    f"UPDATE histone SET description={histone_description_id} WHERE id='cH4_(Animals)'"
)
cursor.execute(query)

In [43]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='cH4_(Animals)'"
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,cH4_(Animals),variant,Metazoa,33208,257,cH4,257,cH4_(Animals) clustered H4 histones in animals...,None,None,...,None,None,The threonine 71 position of histone H4 is hig...,None,None,None,None,None,None,None


In [44]:
# Make sure data is committed to the database
conn.commit()

# To Do H2B

## <span style="color:black">Add a alternate name H2B.E to cH2B.E_(Mus_musculus)</span>

In [45]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cH2B.E_(Mus_musculus)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,89,H2B.21,None,None,None,cH2B.E_(Mus_musculus)


In [46]:
data_alternate_name = {
    "name": "H2B.E",
    "taxonomy": None,
    "gene": None,
    "splice": None,
    "histone": "cH2B.E_(Mus_musculus)",
}
cursor.execute(add_alternate_names, data_alternate_name)

In [47]:
query = (
    "SELECT * FROM alternative_name "
    "WHERE histone='cH2B.E_(Mus_musculus)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,89,H2B.21,None,None,None,cH2B.E_(Mus_musculus)
1,120,H2B.E,None,None,None,cH2B.E_(Mus_musculus)


In [48]:
# Make sure data is committed to the database
conn.commit()

# Close connections

In [35]:
cursor.close()
conn.close()
tunnel.stop()